# `aws` databases

what if you want to do something with data beyond just dumping it into an `s3` bucket?

`aws` has [several database options](https://aws.amazon.com/free/databases-free-tier/), but we're going to talk about two in particular:

1. `rds` (Relational Database Service): several common `sql rdbms` running on managed `ec2` servers
2. `dynamodb`: an `aws`-specific `nosql` database service

there are also several database options that `aws` doesn't support natively, and of those, we will (time permitting) focus on one: `neo4j`

## understanding `sql`, `rdbms`, and `nosql`

from about 2010 to 2016 or so every blogger on the internet wrote an article explaining the difference between `nosql` and `sql` (or specifically: `rdbms`). I won't even pretend to be able to do a better job than them. Here are a few worth reading:

1. [the wikipedia `nosql` entry](https://en.wikipedia.org/wiki/NoSQL)
2. [a good blog post motivating why one might choose `nosql`](https://www.upwork.com/hiring/data/sql-vs-nosql-databases-whats-the-difference/)
3. [a random stack exchange link that 57 people on the internet like](https://dba.stackexchange.com/questions/5/what-are-the-differences-between-nosql-and-a-traditional-rdbms)
4. [another good blog post focusing on the differences between the two](http://www.thegeekstuff.com/2014/01/sql-vs-nosql-db/?utm_source=tuicool)

what follows is about as high-level an introduction to these topics as I can give. If you'd like to know more, you have a lifetime to ~\*~LeArN~\*~!!

### the basics: `sql`

`sql` (Structured Query Language) has existed since the 1970s, been an ANSI ([American National Standards Insitute](https://en.wikipedia.org/wiki/American_National_Standards_Institute)) standard since 1986, and an ISO ([International Organization for Standardization](https://en.wikipedia.org/wiki/International_Organization_for_Standardization)) standard since 1987.

show of hands: who here is older than ANSI-standard `sql`?

you've learned `sql` in 510 and almost certainly used it in data applications. it's everywhere. 

people create perverse monstrocities within real programming language to allow people to write queries in `sql` instead (I'm looking at you, [sqldf](https://cran.r-project.org/web/packages/sqldf/index.html) and [pandassql](https://github.com/yhat/pandasql). you should be ashamed of yourselves)

you cannot escape it.

which is a good thing, I suppose -- having one universally accepted standard for how you interact with large, normalized, relational data means that you only need to learn it once, and large user communities mean that the standards will be both rigorous and evolving.

### the basics: `rdbms`

it is somewhat common for people to *say* `sql` but actually *talk* about `rdbm`s ([Relational DataBase Management Systems](https://en.wikipedia.org/wiki/Relational_database_management_system)). as the `l` in `sql` implies, `sql` is a *language*. it would exist if no database had ever been built.

but databases have been built, and most of them have been built under the same paradigm:

1. tables
2. records
3. links between records (relationships, *not* relations)

*tables* encapsulate single objects or concepts.

for example, a `Person` table

| person_id | person_name |
|-----------|-------------|
| 1         | zach        |

an `Employer` table

| empl_id | empl_name  |
|---------|------------|
| 1       | eri        |
| 2       | georgetown |

and an `EmploymentHistory` table

| empl_hist_id | person_id | empl_id |
|--------------|-----------|---------|
| 1            | 1         | 1       |
| 2            | 1         | 2       |

*records* (rows in those tables) are single instances of that table object

my personal information might exist as a row in the `Person` table, the properites of ERI might exist in the `Employer` table, and my start and (`null`) end date might exist in the `EmploymentHistory` table

*relationships* exist to tie records in one table to records in another

for example, I may have *foreign key* in the `EmploymentHistory` table pointing to records in the `Person` and `Employer` tables

technically speaking, a *relation* is the table schema with all the records together (so, a table).

the `sql` query *language* is used to manipulate or query those tables, but the infrastructure which maintains those tables, records, and relationships (*i.e.* manages the relations in that database) is an `rdmbs`.

[here is a ranking of popularity of various `rdmbs` systems](https://db-engines.com/en/ranking/relational+dbms). the most important to know (in order of implementation popularity):

1. `oracle`
2. `mysql`
3. `mssql`
4. `psql`

#### `acid`ity

one last note on database managment and `rdbms`: you will occasionally hear reference to "ACID" or "ACID"ity. this is an acronymn which describes the best-practice principles of database *transactions* which allow multiple different users to share a single representation of data and not break everything.

those properties are:

1. atomicity: every transaction is atomic, or "all or nothing". no partial commits are allowed
2. consistency: every transaction must take the database from one valid state to another valid state
3. isolation: every transaction should be "isolated" from others until it is committed (information from uncommitted transactions "doesn't exist" outside of that transaction)
4. durability: once you've committed a transaction it *stays* committed, and can be recovered if needed.

I only bring up `acid` because the above constraints happen to get harder to manage if you want to scale your database to many different servers or parallel processes, or if you want things to be *BLAZING FASTTTTTTTTTT*

## the basics: `nosql`

one problem with `rdbms` is that the tables you create must be defined ahead of time -- the object model of the data you are working with is assumed, and if it changes, how will you deal with that?

a second issue is that the above relies heavily on the existence of one central process (the database server) and one "golden copy" of the underlying data. 

If you want to scale out the number of connected clients or the volume of the data itself, you could keep making your centrla process bigger and faster. At some point, though, you'd like to distribute the workload -- have several servers and access whichever one is free, and create new servers to meet demand

as it turns out, there are several reasons why traditional databases are hard to distribute (*e.g.* joining tables is harder, partitioned or segmented data is a major complication).

"`rdbms` are nice and all, but maybe they aren't worth the hassle. what if we could do something that solved *these* problems (easier to scale, flexible data model) by giving up some of the nice parts (`acid`ity, relationships, or even `sql`)..."

and `nosql` is born. `nosql` is a catch-all term for any database system which abandons the tabular storage mechanism of `rdbms` in favor of something which more directly addresses the above issue.

by losing `tables`, we've also necessarily lost `sql`.

really, it would be better called "norel" or "non-relational" but that doesn't sound as `l33t` so I guess I get it.

there are several types of `nosql` databases:

1. key-value: basically a giant hash table / dictionary.
2. columnar: don't store *records*, store *columns*. not every records must be complete
3. document: save "documents" (*e.g.* `json` or `yaml` text blobs) in `collections`. this is effectively a distributed filesystem
4. graph: emphasize relationships between entities over entity definitions
5. *etc*

### why we talked about this

`aws` implements several different *databases*, but it has separate services for them based on whether they are relational or `nosql`. we're going to cover the two `aws` built-in services:

1. `rds`, the `rdbms` option(s)
2. `dynamodb`, the `nosql` option

***<div align="center"><code>DROP joke WHERE is_bad</code></div>***
<div align="center"><img src="https://i.redd.it/u0doli6bk2c01.jpg" width="500px"></div>

# END OF LECTURE

next lecture: [`aws rds`](012_dbs_2_rds.ipynb)